# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
geo_data_0 = pd.read_csv(url)
geo_data_1 = pd.read_csv(url)
geo_data_2 = pd.read_csv(url)

In [3]:
geo_data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
geo_data_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [7]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [9]:
geo_data_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [10]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

### Вывод

- Есть 3 датасета геологоразведки трёх регионов по 100000 строк.
- Пропусков в данных нет.
- Типы данных менять не надо.

## Обучение и проверка модели

### 1 регион

####  Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [12]:
features_valid, features_test, target_valid, target_test = train_test_split(geo_data_0[['f0', 'f1', 'f2']],
                                                                            geo_data_0['product'], 
                                                                            test_size=0.25, random_state=12345)

In [13]:
features_valid.shape, features_test.shape, target_valid.shape, target_test.shape

((75000, 3), (25000, 3), (75000,), (25000,))

#### Обучите модель и сделайте предсказания на валидационной выборке.

In [14]:
model = LinearRegression()

In [15]:
model.fit(features_valid, target_valid)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
predicted = model.predict(features_test)

#### Сохраните предсказания и правильные ответы на валидационной выборке.

In [17]:
region1 = pd.DataFrame(target_test)
region1['predicted'] = predicted
region1['region'] = 1

In [18]:
region1

,product,predicted,region
71751,10.038645,95.894952,1
80493,114.551489,77.572583,1
2655,132.603635,77.892640,1
53233,169.072125,90.175134,1
91141,122.325180,70.510088,1
...,...,...,...
12581,170.116726,103.037104,1
18456,93.632175,85.403255,1
73035,127.352259,61.509833,1
63834,99.782700,118.180397,1


#### Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [19]:
region1[['product', 'predicted']].mean()

product      92.078597
predicted    92.592568
dtype: float64

In [20]:
mse = mean_squared_error(target_test, predicted)
rmse = np.sqrt(mse)
rmse

37.5794217150813

### 2 регион

####  Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [21]:
features_valid, features_test, target_valid, target_test = train_test_split(geo_data_1[['f0', 'f1', 'f2']],
                                                                            geo_data_1['product'], 
                                                                            test_size=0.25, random_state=12345)

In [22]:
features_valid.shape, features_test.shape, target_valid.shape, target_test.shape

((75000, 3), (25000, 3), (75000,), (25000,))

#### Обучите модель и сделайте предсказания на валидационной выборке.

In [23]:
model.fit(features_valid, target_valid)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
predicted = model.predict(features_test)

#### Сохраните предсказания и правильные ответы на валидационной выборке.

In [25]:
region2 = pd.DataFrame(target_test)
region2['predicted'] = predicted
region2['region'] = 2

In [26]:
region2

,product,predicted,region
71751,80.859783,82.663314,2
80493,53.906522,54.431786,2
2655,30.132364,29.748760,2
53233,53.906522,53.552133,2
91141,0.000000,1.243856,2
...,...,...,...
12581,137.945408,136.869211,2
18456,110.992147,110.693465,2
73035,137.945408,137.879341,2
63834,84.038886,83.761966,2


#### Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [27]:
region2[['product', 'predicted']].mean()

product      68.723136
predicted    68.728547
dtype: float64

In [28]:
mse = mean_squared_error(target_test, predicted)
rmse = np.sqrt(mse)
rmse

0.893099286775616

### 3 регион

####  Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [29]:
features_valid, features_test, target_valid, target_test = train_test_split(geo_data_2[['f0', 'f1', 'f2']],
                                                                            geo_data_2['product'], 
                                                                            test_size=0.25, random_state=12345)

In [30]:
features_valid.shape, features_test.shape, target_valid.shape, target_test.shape

((75000, 3), (25000, 3), (75000,), (25000,))

#### Обучите модель и сделайте предсказания на валидационной выборке.

In [31]:
model.fit(features_valid, target_valid)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [32]:
predicted = model.predict(features_test)

#### Сохраните предсказания и правильные ответы на валидационной выборке.

In [33]:
region3 = pd.DataFrame(target_test)
region3['predicted'] = predicted
region3['region'] = 3

In [34]:
region3

,product,predicted,region
71751,61.212375,93.599633,3
80493,41.850118,75.105159,3
2655,57.776581,90.066809,3
53233,100.053761,105.162375,3
91141,109.897122,115.303310,3
...,...,...,...
12581,28.492402,78.765887,3
18456,21.431303,95.603394,3
73035,125.487229,99.407281,3
63834,99.422903,77.779912,3


#### Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [35]:
region3[['product', 'predicted']].mean()

product      94.884233
predicted    94.965046
dtype: float64

In [36]:
mse = mean_squared_error(target_test, predicted)
rmse = np.sqrt(mse)
rmse

40.02970873393434

### Вывод

1. Cредний запас сырья:
- первый регион предсказано - 92.59, правильный - 92.08
- второй регион предсказано - 68.73, правильный - 68.72
- третий регион предсказано - 94.97, правильный - 94.88
2. RMSE модели:
- в первом регионе равен 37.57
- во втором регионе равен 0.89
- в третьем регионе равен 40.03
3. Больше всего запасов сырья в третьем регионе, но у него и RMSE самый большой, во втором регионе меньше всего запасов и очень маленький RMSE.

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраните в отдельных переменных.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

In [37]:
BUDGET = 10000000000

выбирают 200 лучших точек для разработки, тогда бюджет одной скважины равен:

In [38]:
WELL_BUDGET = BUDGET / 200
WELL_BUDGET

50000000.0

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [39]:
BARREL_INCOME = 450000

### Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

In [40]:
break_even_volume = WELL_BUDGET / BARREL_INCOME
break_even_volume

111.11111111111111

In [41]:
region3['predicted'].mean(), region2['predicted'].mean(), region1['predicted'].mean()

(94.96504596800489, 68.728546895446, 92.59256778438038)

### Напишите выводы по этапу подготовки расчёта прибыли.

Ни один регион в среднем не имеет достаточный объём сырья для безубыточной разработки новых скважин.

### Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
1. Выберите скважины с максимальными значениями предсказаний.
2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
3. Рассчитайте прибыль для полученного объёма сырья.

In [42]:
def profit(region):
    return sum(region.sort_values(by='predicted', ascending=False)['product'].head(200)) * BARREL_INCOME

## Расчёт прибыли и рисков 

- Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [51]:
data_region = {}
for region in [region1, region2, region3]:
    values = []
    for i in range(1000):
        region_subsample = region.sample(n=500, replace=False) 
        values.append(profit(region_subsample))
    values = pd.Series(values)
    data_region['region'+str(region['region'][1])] = [sum(values) / 1000 - BUDGET, values.quantile(0.025) - BUDGET, values.quantile(0.975) - BUDGET, (values - BUDGET < 0).mean()]

In [52]:
pd.DataFrame.from_dict(data_region).T.rename(columns={0:'mean_profit', 1:'2.5%_quantile', 2:'97.5%_quantile', 3:'risk'})

,mean_profit,2.5%_quantile,97.5%_quantile,risk
region1,3.787134e+08,-1.635807e+08,8.661495e+08,0.074
region2,4.513089e+08,7.524404e+07,8.387828e+08,0.014
region3,3.949220e+08,-1.368253e+08,8.917989e+08,0.070


- Учитывая 2.5%_quantile лучше выбрать второй регион, так как в случаи риска он остаётся прибыльным, в отличие от остальных регионов.
- Также самый маленький риск убытков во втором регионе состовляет 1.4%, в первом равен 7.4% и в третьем 7%.

## Вывод

1. Cредний запас сырья:
- первый регион предсказано - 92.59, правильный - 92.08
- второй регион предсказано - 68.73, правильный - 68.72
- третий регион предсказано - 94.97, правильный - 94.88
2. RMSE модели:
- в первом регионе равен 37.57
- во втором регионе равен 0.89
- в третьем регионе равен 40.03
3. Больше всего запасов сырья в третьем регионе, но у него и RMSE самый большой, во втором регионе меньше всего запасов и очень маленький RMSE.
4. Ни один регион в среднем не имеет достаточный объём сырья для безубыточной разработки новых скважин.
5. Учитывая 2.5%_quantile лучше выбрать второй регион, так как в случаи риска он остаётся прибыльным, в отличие от остальных регионов.
6. Также самый маленький риск убытков во втором регионе состовляет 1.4%, в первом равен 7.4% и в третьем 7%.